## Requirement

(Optional) Using Generation by Syntax Tree and PAIP 151-167. Generate Sentence by Grammar. Implement a dialogue system, you may implement a mock system like West World, or Prison Break, etc.

- Using AIML markup language to generate same pattern. 

- Think what markup language or rule based can do, what they cannot do?

Bonus: If you finish this task, please connect with me. I will send you a **mysterious gift**.

### References:
a. PAIP https://github.com/Artificial-Intelligence-for-NLP/References/blob/master/AI%20%26%20Machine%20Learning/Paradigms-of-Artificial-Intelligence-Programming%20copy.pdf

b. https://www.wikiwand.com/en/ELIZA

c. https://www.eclecticenergies.com/ego/eliza

d. https://baike.baidu.com/item/%E8%89%BE%E4%B8%BD%E8%8E%8E/9030600

# chapter 5: ELIZA: Dialog with a Machine
## 5.1 Describing and Specifying ELIZA
Now that we have an idea of what ELIZA is like, we can begin the description and spec- ificationof the program, and eventually move to the implementation and debugging. The ELIZA algorithm can be described simply as: 

- (1)read an input, 

- (2) find a pattern that matches the input, 

- (3)transform the input into a response, and 

- (4)print the response. These four steps are repeated for each input.

The specificationand implementation of steps (1)and (4)are trivial: for (I),use the built-in read function to read a list of words, and for (4)use pri n t to print the list of words in the response.
Of course, there are some drawbacks to this specification. The user will have to type a real list- using parentheses- and the user can't use characters that are special to read, like quotation marks, commas, and periods. So our input won't be as unconstrained as in the sample dialog, but that's a small price to pay for the convenience of having half of the problem neatly solved.


## 1. parenthesis parser

In [1]:
import re, debug_tools
from collections import defaultdict

# @debug_tools.debug_print
def parenthesis_parser(string):
    string = [s.strip() for s in re.split('([\(\)\ ])', string) if s not in ('', ' ', '\'')]
#     print(string)
    stack_level = 0
    result = defaultdict(list)
    
    for s in string:
        if s == '(': 
            stack_level += 1         
        elif s == ')':
            result[stack_level-1].append(tuple(result.pop(stack_level)))
            stack_level -= 1          
        elif s!='': result[stack_level].append(s)
#         print(result)
            
    return result[0]


### 2.1 pattern matching
```lisp
(defun pat-match (pattern input)
  (if (variable-p pattern)
    t
    (if (or (atom pattern ) (atom input))
      (eql pattern input)
      (and (pat-match (car pattern) (car input))
           (pat-match (cdr pattern) (cdr input))))))           
           
(defun variable-p (x)
  (and (symbolp x) (equal (char (symbol-name x) 0) #\?)))
```
**input** is the key word of python, here use **_input**

In [2]:
# @debug_tools.debug_print
def atom(x:str):
    return type(x) == str

# @debug_tools.debug_print
def variable_p(x:str):
    return x[0] == '?'

@debug_tools.debug_print
def pat_match(pattern:list, _input:list):
    #"Does pattern match input? Any variable can match anything."
    if (pattern == _input) or variable_p(pattern):
        return True
    else:
        if atom(pattern) or atom(_input):
            return pattern == _input
        else:
            return pat_match(pattern[0], _input[0]) and pat_match(pattern[1:], _input[1:])

test_cases1 = """
'(i need a ?X) '(i need a vacation)
'(i need a ?X) '(i really need a vacation)
'(this is easy) '(this is easy)
'(?X is ?X) '((2 + 2) is 4)
"""

def test_pat_match(pat_match, test_cases):
    debug_tools.stack_level = 0
    for test_case in test_cases[1:-1].split('\n'):
        pattern, _input = parenthesis_parser(test_case)
        print('\n=====================================================================')
        print('pattern:\t', pattern)
        print('input:\t\t', _input)
        print('result:\t\t', pat_match(pattern, _input))

test_pat_match(pat_match, test_cases1)


pattern:	 ('i', 'need', 'a', '?X')
input:		 ('i', 'need', 'a', 'vacation')
|s: pat_match ((('i', 'need', 'a', '?X'), ('i', 'need', 'a', 'vacation')), {})
||s: pat_match (('i', 'i'), {})
||e: pat_match (('i', 'i'), {}) = True
||s: pat_match ((('need', 'a', '?X'), ('need', 'a', 'vacation')), {})
|||s: pat_match (('need', 'need'), {})
|||e: pat_match (('need', 'need'), {}) = True
|||s: pat_match ((('a', '?X'), ('a', 'vacation')), {})
||||s: pat_match (('a', 'a'), {})
||||e: pat_match (('a', 'a'), {}) = True
||||s: pat_match ((('?X',), ('vacation',)), {})
|||||s: pat_match (('?X', 'vacation'), {})
|||||e: pat_match (('?X', 'vacation'), {}) = True
|||||s: pat_match (((), ()), {})
|||||e: pat_match (((), ()), {}) = True
||||e: pat_match ((('?X',), ('vacation',)), {}) = True
|||e: pat_match ((('a', '?X'), ('a', 'vacation')), {}) = True
||e: pat_match ((('need', 'a', '?X'), ('need', 'a', 'vacation')), {}) = True
|e: pat_match ((('i', 'need', 'a', '?X'), ('i', 'need', 'a', 'vacation')), {}) = 

### 2.2 pattern matching with bindings
```lisp
(defun pat-match (pattern input &optional (bindings no-bindings))
  "Match pattern against input in the context of the bindings"
  (cond ((eq bindings fail) fail)
        ((variable-p pattern) (match-variable pattern input bindings))
        ((eql pattern input) bindings)
        ((and (consp pattern) (consp input)) 
          (pat-match (cdr pattern) (cdr input) (pat-match (car pattern) (car input) bindings)))
          
        (t fail)
        
    ))

(defun match-variable (var input bindings)
  "Does VAR match input? Uses (or updates) and returns bindings."
  (let ((binding (get-binding var bindings)))
    (cond ((not binding) (extend-bindings var input bindings))
          ((equal input (binding-val binding)) bindings)
          (t fail))))
          
(defconstant fail nil)

(defun get-binding (var bindings)
  "Find a (variable . value) pair i n a binding list." 
  (assoc var bindings)
    
(defun binding-val (binding)
  "Get the value part of a single binding." 
  (cdr binding))
    
(defun lookup (var bindings)
  "Get the value part (for var) from a binding list." 
  (binding-val (get-binding var bindings)))
    
(defun extend-bindings (var val bindings
  "Add a (var . value) pair to a binding list." 
  (cons (cons var val) bindings))

```

In [3]:
import functools
import debug_tools

def variable_p(x:str):
    return type(x)==str and x[0] == '?'


def get_binding(var:str, bindings:dict):
    if var in bindings:
        return (var, bindings[var])
    else:
        return False

def binding_val(binding:tuple):
    return binding[1]

def lookup(var:str, bindings:dict):
    return binding_val(get_binding(var, bindings))

def extend_bindings(var:str, val:str, bindings:dict):
    bindings[var] = val
    return bindings
    

def consp(x:tuple):
    return type(x) == tuple

# @debug_tools.debug_print
def match_variable(var:str, _input:tuple, bindings:dict):
    binding = get_binding(var, bindings)
    
#     print(_input, binding_val(binding))
    
    if not binding:
        extend_bindings(var, _input, bindings=bindings)
        return bindings

    elif _input == binding_val(binding):
        return bindings
    
    else:
        return False


@debug_tools.debug_print
def pat_match(pattern:tuple, _input:tuple, bindings=dict()):
    #"Does pattern match input? Any variable can match anything."
    if bindings == False:
        return False
    
    elif variable_p(pattern):
        return match_variable(pattern, _input, bindings=bindings)
    
    elif (pattern == _input):
        return bindings
    
    elif consp(pattern) and consp(_input):
        return pat_match(pattern[1:], _input[1:], bindings=pat_match(pattern[0], _input[0], bindings=bindings))
    
    else:
        return False
    
test_cases2 = '''
'(?X is ?X) '((2 + 2) is 4)
'(?X is ?X) '((2 + 2) is (2 + 2))
'(?P need a ?X) ' ( i need a vacation)
'(?P need . ?X) ' ( i need a long vacation)
'''

def test_pat_match(pat_match, test_cases:str):
    debug_tools.stack_level = 0
    for test_case in test_cases[1:-1].split('\n'):
        if not test_case: continue
        pattern, _input = parenthesis_parser(test_case)
        print('\n=====================================================================')
        print('pattern:\t', pattern)
        print('input:\t\t', _input)
        print('result:\t\t', pat_match(pattern, _input, bindings=dict())) #蜜汁bug

debug_tools.is_debug = True
test_pat_match(pat_match, test_cases2)
print('\n\n=====================================================================')
print('full test set')
print('=====================================================================')
debug_tools.is_debug = False
test_pat_match(pat_match, test_cases1)


pattern:	 ('?X', 'is', '?X')
input:		 (('2', '+', '2'), 'is', '4')
|s: pat_match ((('?X', 'is', '?X'), (('2', '+', '2'), 'is', '4')), {'bindings': {}})
||s: pat_match (('?X', ('2', '+', '2')), {'bindings': {}})
||e: pat_match (('?X', ('2', '+', '2')), {'bindings': {'?X': ('2', '+', '2')}}) = {'?X': ('2', '+', '2')}
||s: pat_match ((('is', '?X'), ('is', '4')), {'bindings': {'?X': ('2', '+', '2')}})
|||s: pat_match (('is', 'is'), {'bindings': {'?X': ('2', '+', '2')}})
|||e: pat_match (('is', 'is'), {'bindings': {'?X': ('2', '+', '2')}}) = {'?X': ('2', '+', '2')}
|||s: pat_match ((('?X',), ('4',)), {'bindings': {'?X': ('2', '+', '2')}})
||||s: pat_match (('?X', '4'), {'bindings': {'?X': ('2', '+', '2')}})
||||e: pat_match (('?X', '4'), {'bindings': {'?X': ('2', '+', '2')}}) = False
||||s: pat_match (((), ()), {'bindings': False})
||||e: pat_match (((), ()), {'bindings': False}) = False
|||e: pat_match ((('?X',), ('4',)), {'bindings': {'?X': ('2', '+', '2')}}) = False
||e: pat_match ((('i

### 2.3 Segment Pattern Matching

```lisp
(defun pat-match (pattern input &optional (bindings no-bindings))
  (cond ((eq bindings fail) fail)
        ((variable-p pattern) 
         (match-variable pattern input bindings))
        ((eql pattern input) bindings)
        ((segment-pattern-p pattern)
         (segment-match pattern input bindings))
        ((and (consp pattern) (consp input))
         (pat-match (cdr pattern) (cdr input)
                    (pat-match (car pattern) (car input) bindings)))
        (t fail)))

(defun segment-pattern-p (pattern)
  (and (consp pattern)
       (starts-with (car pattern) '?*)))

(defun starts-with (lst symb)
  (if (consp lst)
    (eql (car lst) symb)
    (eql lst symb)))

(defun segment-match (pattern input bindings &optional (start 0))
  (let ((var (cadr (car pattern)))
        (pat (cdr pattern)))
    (if (null pat)
      (match-variable var input bindings)
      (let ((pos (position (car pat) input :start start :test #'equal)))
        (if (null pos)
          fail
          (let ((b2 (pat-match pat (subseq input pos) bindings)))
            (if (eq b2 fail)
              (segment-match pattern input bindings (1+ pos))
              (match-variable var (subseq input 0 pos) b2))))))))
```

In [4]:
@debug_tools.debug_print
def pat_match(pattern:tuple, _input:tuple, bindings=dict()):
    #"Does pattern match input? Any variable can match anything."
    if bindings == False:
        return False
    
    elif variable_p(pattern):
        return match_variable(pattern, _input, bindings=bindings)
    
    elif (pattern == _input):
        return bindings
    
    elif segment_pattern_p(pattern):
        return segment_match(pattern, _input, bindings)
    
    elif consp(pattern) and consp(_input):
        return pat_match(pattern[1:], _input[1:], bindings=pat_match(pattern[0], _input[0], bindings=bindings))
    
    else:
        return False
    
def segment_pattern_p(pattern:tuple):
    return consp(pattern) and start_with(pattern[0], '?*')

def start_with(pattern:tuple, symb:str):
    if consp(pattern):
        return pattern[0] == symb
    else:
        return pattern == symb

# @debug_tools.debug_print
def position(var:str, lst:tuple, start:int):
    lst = lst[start:]
    if var in lst:
        return lst.index(var) + start
    else: 
        return None
    
@debug_tools.debug_print
def segment_match(pattern:tuple, _input:tuple, bindings:dict, start=0):
    var = pattern[0][1]
    pat = pattern[1:]

    if not pat:
        return match_variable(var, _input, bindings=bindings)
    else:
        pos = position(pat[0], _input, start)
#         print(pos, pat[0], _input)
        if pos == None:
            return False
        else:
            b2 = pat_match(pat, _input[pos:], bindings=bindings)
            if b2 == False:
                return segment_match(pattern, _input, bindings=bindings, start=pos+1)
            else:
                return match_variable(var, tuple(_input[0:pos]), bindings=b2)

            
            
test_cases3 = '''
'((?* ?p) need (?* ?X)) '(Mr Hulot and I need a vacation)
'((?* ?X) is a (?* ?y)) '(what he is is a fool)
'((?* ?X) a b (?* ?X)) '(1 2 a b a b 1 2 a b)
'''

debug_tools.is_debug = True
test_pat_match(pat_match, test_cases3)
print('\n\n=====================================================================')
print('full test set')
print('=====================================================================')
debug_tools.is_debug = False
test_pat_match(pat_match, test_cases1+test_cases2)


pattern:	 (('?*', '?p'), 'need', ('?*', '?X'))
input:		 ('Mr', 'Hulot', 'and', 'I', 'need', 'a', 'vacation')
|s: pat_match (((('?*', '?p'), 'need', ('?*', '?X')), ('Mr', 'Hulot', 'and', 'I', 'need', 'a', 'vacation')), {'bindings': {}})
||s: segment_match (((('?*', '?p'), 'need', ('?*', '?X')), ('Mr', 'Hulot', 'and', 'I', 'need', 'a', 'vacation'), {}), {})
|||s: pat_match ((('need', ('?*', '?X')), ('need', 'a', 'vacation')), {'bindings': {}})
||||s: pat_match (('need', 'need'), {'bindings': {}})
||||e: pat_match (('need', 'need'), {'bindings': {}}) = {}
||||s: pat_match (((('?*', '?X'),), ('a', 'vacation')), {'bindings': {}})
|||||s: segment_match (((('?*', '?X'),), ('a', 'vacation'), {}), {})
|||||e: segment_match (((('?*', '?X'),), ('a', 'vacation'), {'?X': ('a', 'vacation')}), {}) = {'?X': ('a', 'vacation')}
||||e: pat_match (((('?*', '?X'),), ('a', 'vacation')), {'bindings': {'?X': ('a', 'vacation')}}) = {'?X': ('a', 'vacation')}
|||e: pat_match ((('need', ('?*', '?X')), ('need', '

### 2.4 segment matching enhancement
```lisp
(defun segment-match (pattern input bindings &optional (start 0))
  (let ((var (cadr (car pattern)))
        (pat (cdr pattern)))
    (if (null pat)
      (match-variable var input bindings)
      (let ((pos (position (car pat) input :start start :test #'equal)))
        (if (null pos)
          fail
          (let ((b2 (pat-match pat (subseq input pos) 
                     bindings)))
                     
            (if (eq b2 fail)
              (segment-match pattern input bindings (1+ pos))
              (match-variable var (subseq input 0 pos) b2))))))))

(defun segment-match (pattern input bindings &optional (start 0))
  (let ((var (cadr (car pattern)))
        (pat (cdr pattern)))
    (if (null pat)
      (match-variable var input bindings)
      (let ((pos (position (car pat) input :start start :test #'equal)))
        (if (null pos)
          fail
          (let ((b2 (pat-match pat (subseq input pos)
                     (match-variable var (subseq input 0 pos) bindings))))
                     
            (if (eq b2 fail)
              (segment-match pattern input bindings (1+ pos)) 
               b2)))))))
              
```

In [5]:
## this is a buggy version
# @debug_tools.debug_print
# def segment_match(pattern:tuple, _input:tuple, bindings:dict, start=0):
#     var = pattern[0][1]
#     pat = pattern[1:]

#     if not pat:
#         return match_variable(var, _input, bindings=bindings)
#     else:
#         pos = position(pat[0], _input, start)
#         if pos == None:
#             return False
#         else: # different from here
#             b2 = pat_match(pat, _input[pos:], bindings=match_variable(var, _input[0:pos], bindings))
#             if b2 == False:
#                 return segment_match(pattern, _input, bindings=bindings, start=pos+1)
#             else:
#                 return b2
            
# debug_tools.is_debug = True
# test_pat_match(pat_match, test_cases3)
# print('\n\n=====================================================================')
# print('full test set')
# print('=====================================================================')
# debug_tools.is_debug = False
# test_pat_match(pat_match, test_cases1+test_cases2)

## 3. The ELIZA Program: A Rule-Based Translator

```lisp
(defun eliza ()
  (loop
    (print 'eliza>)
    (write (flattern (use-eliza-rule (read))) :pretty t)))

(defun use-eliza-rule (input)
  (some #'(lambda (rule)
            (let ((result (pat-match (rule-pattern rule) input)))
              (if (not (eq result fail))
                (sublis (switch-viewpoint result)
                        (random-elt (rule-responses rule))))))
        *eliza-rules*))
(defun switch-viewpoint (words)
  (sublis '((i . you) (you . i) (me . you) (am . are))
          words))

(defun random-elt (lst)
  (elt lst (random (length lst))))

(defun mappend (fn lst)
  (apply #'append (mapcar fn lst)))

(defun flattern (lst)
  (mappend #'mklist lst))

(defun mklist (x)
  (if (listp x)
    x
    (list x)))

(eliza)

```

In [6]:
import random

def use_eliza_rule(_input:list):
    global eliza_rules  
    _input = tuple(_input.split(' '))
    for pattern in eliza_rules:    
        result = pat_match(pattern, _input, bindings=dict())
        if result != False:
            return sublis(switch_viewpoint(result), random.choice(eliza_rules[pattern]))    
    return '#No pattern detected#'
        
def switch_viewpoint(bindings:dict):
    def switch(words:tuple):
        switch_words = {'i':'you', 'you':'i', 'me':'you', 'am':'are'}
        return [switch_words[w] if w in switch_words else w for w in words]  
    return {pattern: switch(value) for pattern, value in bindings.items()}

@debug_tools.debug_print
def sublis(bindings:dict, pattern:tuple):
    if type(pattern)==str:
        if pattern in bindings:
            return ' '.join(bindings[pattern])
        else:
            return pattern
    else: 
        return ' '.join([sublis(bindings, e) for e in pattern if e!='?*'])

rules = '''
  '(((?* ?x) hello (?* ?y))
     (how do you do. Please state your problem.))
    (((?* ?x) i want (?* ?y))
     (what would it mean if you got ?y)
     (why do you want ?y))
    (((?* ?x) if (?* ?y))
     (do you really think its likely that ?y)
     (what do you think about ?y)
     (really-- if ?y))
    (((?* ?x) no (?* ?y))
     (why not?)
     (you are being a bit negative)
     (are you saying "NO" just to be negative?))
    (((?* ?x) i feel (?* ?y))
     (do you often feel ?y ?))
    (((?* ?x) i felt (?* ?y))
     (what other feelings do you have?))
'''

def get_eliza_rules(rules):
    eliza_rules = dict()
    for e in parenthesis_parser(rules):
        pattern = e[0]
        responses = e[1:]
        eliza_rules[pattern] = responses
    return eliza_rules
    
eliza_rules = get_eliza_rules(rules)

test_cases = '''
elza hello
today is a good day, i want to talk with you
mr gao will give me a mystical bonus if i create you correctly
no it just a joke
you are stupid i feel sad
i felt better now
'''

debug_tools.is_debug = False
for _input in test_cases.split('\n'):
    print('input:\t',_input)
    print('eliza:\t',use_eliza_rule(_input))
    print()

input:	 
eliza:	 #No pattern detected#

input:	 elza hello
eliza:	 how do you do. Please state your problem.

input:	 today is a good day, i want to talk with you
eliza:	 what would it mean if you got to talk with i

input:	 mr gao will give me a mystical bonus if i create you correctly
eliza:	 what do you think about you create i correctly

input:	 no it just a joke
eliza:	 are you saying "NO" just to be negative?

input:	 you are stupid i feel sad
eliza:	 do you often feel sad ?

input:	 i felt better now
eliza:	 what other feelings do you have?

input:	 
eliza:	 #No pattern detected#



### 3.1 User interface

In [7]:
def eliza():
    print('ELIZA PROGRAM')
    _input = ''
    while _input != 'exit':
        _input = input('input:\t')
        print('eliza:\t', use_eliza_rule(_input))

# eliza()

## 4. Final version
run following code:
```shell
python3 eliza.py
```